In [37]:
from qiskit_algorithms import Grover
# from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
from qiskit_algorithms import AmplificationProblem
from qiskit import transpile, ClassicalRegister, QuantumRegister
from qiskit.transpiler import CouplingMap
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
import gen_utilities as utl
importlib.reload(utl)
os.chdir(current_dir)
import math
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as AerSampler
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
from qiskit.visualization import plot_histogram
import json
import pickle
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke, FakeTorino
import qiskit
from qiskit.providers.options import Options
from qiskit_ibm_runtime.options.twirling_options import TwirlingOptions
from qiskit_ibm_runtime.options import SamplerOptions
from qiskit_ibm_runtime import SamplerV2
from qiskit import QuantumCircuit
from qiskit.converters import (
    circuit_to_dag, 
    dag_to_circuit
    )
from copy import deepcopy
from qiskit.circuit.library import TGate, HGate, ZGate, SXGate, XGate, SGate, YGate, PhaseGate #QFTGate
from qiskit.synthesis import SolovayKitaevDecomposition
from qiskit.synthesis import generate_basic_approximations
from qiskit.transpiler.passes import SolovayKitaev

from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import phase_estimation
# from qiskit.circuit.library import QFT
from qiskit.synthesis import synth_qft_full

print(qiskit.__version__)

# BASIS=["cx", "rz", "h"]
# BASIS=["cx", "h", "s", "t"]

SHOTS=200000

def meas_qubits(circ: QuantumCircuit, target_qubits):
    '''Meas the desired qubits. Gives the meas label.'''
    meas_cregs=ClassicalRegister(len(target_qubits), "meas")
    circ.add_register(meas_cregs)
    for idx, q in enumerate(target_qubits):
        circ.measure(q, meas_cregs[idx])

def create_noise_est_cir(circ):
    noise_est_circ=utl.replace_gates_h_sx_rx(circ)
    return noise_est_circ

def replace_t_with_injection(circ: QuantumCircuit, num_qubits: int):
    '''Replaces t gate with injection.'''
    dag=circuit_to_dag(circ)
    throwaway_cr=ClassicalRegister(1, "throw_away")
    ancilla_qr=QuantumRegister(1)
    new_qc=deepcopy(circ)
    new_qc.data=[]
    new_qc.add_register(throwaway_cr)
    new_qc.add_register(ancilla_qr)
    op_nodes=dag.op_nodes()
    for node in op_nodes:
        # print(node.name)
        if node.name=="t" or node.name=="tdg":
            new_qc.h(ancilla_qr[0])
            if node.name=="t":
                new_qc.t(ancilla_qr[0])
            else:
                new_qc.tdg(ancilla_qr[0])
            new_qc.cx(node.qargs[0]._index, ancilla_qr[0])
            if node.name=="t":
                new_qc.cs(ancilla_qr[0], node.qargs[0]._index)
            else:
                new_qc.csdg(ancilla_qr[0], node.qargs[0]._index)
            new_qc.measure(ancilla_qr[0], throwaway_cr[0])
            new_qc.reset(ancilla_qr[0])

        else:
            utl.copy_node(new_qc, node)
    return new_qc

def split_circuit(circuit, start, end):
    nq = len(circuit.qubits)
    qc2 = QuantumCircuit(nq)
    for x in circuit[start:end]:
        qc2.append(x[0], x[1])
    return qc2

def ghz(num_qubits):
    circ=QuantumCircuit(num_qubits)
    circ.h(0)
    # for idx in list(range(num_qubits))[1::]:
    #     circ.cx(0, idx)
    for idx in list(range(num_qubits))[:-1:]:
        circ.cx(idx, idx+1)
    return circ

def qpe(targets, angle):
    qpe3 = QuantumCircuit(targets)

    # Apply H-Gates to counting qubits:
    for qubit in range(targets-1):
        qpe3.h(qubit)

    # Prepare our eigenstate |psi>:
    qpe3.x(targets-1)

    # Do the controlled-U operations:
    repetitions = 1
    for counting_qubit in range(targets-1):
        for i in range(repetitions):
            qpe3.cp(angle, counting_qubit, targets-1)
        repetitions *= 2

    # Do the inverse QFT:
    # print(QFT(targets-1, inverse=True))
    qpe3 = qpe3.compose(synth_qft_full(targets-1, inverse=True, do_swaps=True), range(targets-1))
    return qpe3

1.3.0


In [38]:
from qiskit.result import Result
from qiskit_ibm_runtime import QiskitRuntimeService

# service=QiskitRuntimeService()
# backend=service.backend("ibm_torino")
p1_error_prob=0.001
p2_error_prob=10*p1_error_prob
p1_depol=depolarizing_error(p1_error_prob, 1)
p2_depol=depolarizing_error(p2_error_prob, 2)
noise_model=NoiseModel()
noise_model.add_all_qubit_quantum_error(p1_depol, ["measure", "x", "rz", "sx", "reset"])
noise_model.add_all_qubit_quantum_error(p2_depol, ["cz"])

noisy_sim=AerSimulator(noise_model=noise_model)
# noisy_sim=AerSampler(options=dict(run_options=dict(noise_model=noise_model)))
# backend=FakeTorino()
# noisy_sim=SamplerV2(backend)

In [ ]:
# Ideal execution
BASIS=["cz", "x", "rz", "sx"]

target="1"*5
num_qubits=len(target)
# circ=qpe(num_qubits, math.pi)
oracle=Statevector.from_label(target)
problem = AmplificationProblem(oracle, is_good_state=[target])
print(problem.is_good_state(target))
# grover_op=problem.grover_operator
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(1 / 2**num_qubits)))
)
# optimal_num_iterations=1
circ=Grover().construct_circuit(problem, optimal_num_iterations)
meas_qubits(circ, list(range(num_qubits)))
circ=transpile(circ, basis_gates=["cx", "u"], optimization_level=3)
# basic_approx=generate_basic_approximations(basis_gates=[TGate(), HGate(), SGate()], depth=3)
# skd=SolovayKitaev(recursion_degree=2, basic_approximations=basic_approx)
# circ=skd(circ)
print(circ.draw(fold=-1))
ideal_circ=deepcopy(circ)

ideal_sim=AerSimulator()
result_ideal=ideal_sim.run(ideal_circ, shots=SHOTS).result()
ideal_counts=result_ideal.get_counts()
print(ideal_counts)
ideal_dist={k:v/SHOTS for k, v in ideal_counts.items()}


True
global phase: 5.1603
        ┌────────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐     ┌───┐ ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           ┌─────────────┐                                                             

In [40]:
# circ=transpile(circ, optimization_level=3, basis_gates=BASIS)
# coupling=CouplingMap(list(zip(list(range(num_qubits)), list(range(1, num_qubits+1)))))
# circ=transpile(circ, coupling_map=coupling, initial_layout=list(range(num_qubits)), seed_transpiler=0, optimization_level=0)
# print(circ.draw(fold=-1))
# circ=replace_t_with_injection(circ, num_qubits)
print(circ.count_ops())
print(circ.draw(fold=-1))
circ=transpile(circ, seed_transpiler=0, optimization_level=3, basis_gates=BASIS)
noise_est_circ=create_noise_est_cir(circ)
clifford_noise_est_cir=utl.circ_replace_rz(noise_est_circ, num_qubits)
clifford_noise_est_cir=utl.remove_idle_wires(clifford_noise_est_cir)
print(f"noise payload circ {circ.draw(fold=-1)}")
print(f"noise estimation circ {noise_est_circ.draw(fold=-1)}")
print("ops payload: ", circ.count_ops())
print("ops noise est cir: ", noise_est_circ.count_ops())
print("ops clifford noise est cir: ", clifford_noise_est_cir.count_ops())
      

OrderedDict({'u': 305, 'cx': 256, 'measure': 5})
global phase: 5.1603
        ┌────────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐     ┌───┐ ┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           ┌─────────────┐                 

In [41]:
result_cliff_noise_est=ideal_sim.run(clifford_noise_est_cir, shots=SHOTS).result()
result_noisy=noisy_sim.run([circ, noise_est_circ], shots=SHOTS).result()


In [42]:
def strip_classical_reg(remove_pos: int, dist: dict):
    '''Modifies the keys of dist by splitting it the key by space and removing the strings at index remove_pos.
    This is from stripping unwanted classical registers.'''
    new_dict={}
    for k, v in dist.items():
        k=k.split(" ")
        k.pop(remove_pos)
        k="".join(k)
        count=new_dict.get(k, 0)
        count+=v
        new_dict[k]=count
    return new_dict

In [43]:
# print(result_noisy)
# counts_ideal_noise_est=strip_classical_reg(0, result_cliff_noise_est.get_counts())
# noisy_counts=strip_classical_reg(0, result_noisy.get_counts()[0])
# counts_noisy_noise_est=strip_classical_reg(0, result_noisy.get_counts()[1])

counts_ideal_noise_est=result_cliff_noise_est.get_counts()
noisy_counts=result_noisy.get_counts()[0]
counts_noisy_noise_est=result_noisy.get_counts()[1]

print(noisy_counts)
print(counts_noisy_noise_est)
dist_noisy_noise_est={k:v/SHOTS for k,v in counts_noisy_noise_est.items()}
print(counts_ideal_noise_est)

{'11011': 7373, '01100': 5666, '01111': 7868, '00110': 5020, '11111': 19376, '01000': 5616, '11001': 6319, '00100': 4878, '00111': 5621, '11101': 7544, '10111': 6342, '00000': 4787, '00010': 4972, '01011': 6091, '00011': 5117, '11000': 6113, '01110': 5948, '11010': 6343, '00001': 4758, '10010': 5275, '01101': 5847, '11110': 7874, '10101': 5423, '11100': 6443, '10100': 5157, '00101': 4986, '10110': 5684, '10001': 5188, '01010': 5720, '10000': 5345, '10011': 5686, '01001': 5620}
{'11111': 27114, '10011': 5958, '01011': 7273, '00011': 3581, '11011': 11770, '10001': 3570, '11010': 6666, '10000': 2430, '11101': 11931, '10100': 3664, '00010': 2239, '01101': 6511, '11100': 7304, '00110': 3432, '10110': 6209, '00001': 2271, '01000': 2602, '01111': 11695, '10111': 10684, '11110': 12150, '10101': 5972, '00101': 3451, '01001': 3988, '00111': 5597, '01010': 4002, '11000': 4111, '11001': 6486, '10010': 3581, '00100': 2221, '01110': 6152, '01100': 3915, '00000': 1470}
{'11111': 200000}


In [44]:
with open("ideal_counts_sim.json", "w") as f:
    json.dump(ideal_counts, f)

with open("noisy_noise_est_dist_sim.json", "w") as f:
    json.dump(dist_noisy_noise_est, f)

with open("ideal_noise_est_counts_sim.json", "w") as f:
    json.dump(counts_ideal_noise_est, f)

with open("noisy_counts_sim.json", "w") as f:
    json.dump(noisy_counts, f)

In [45]:
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'cz', 'id', 'rz', 'sx', 'x']
  Instructions with noise: ['sx', 'measure', 'rz', 'x', 'cz', 'reset']
  All-qubits errors: ['measure', 'x', 'rz', 'sx', 'reset', 'cz']
